In [19]:
"""
Data Preprocessing
"""

import os
import re
import pandas as pd

# read train data
file_dir = '/Users/linzhizhang/Stu/SM4/90051/data'
file_name = "train.txt"

def parse(data:list)->list: 
    # Python Notation: https://www.python.org/dev/peps/pep-3107/
    """
    Prase each line of the data
    Return a nested list, such as [[id1, id2, id2,...],...]
    """
    parsed_data = []
    for line in data:
        # remove \n at the end of each line
        line = re.sub(r"(?<=\d)\n", "", line)
        # split IDs by \t
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        parsed_data.append(line)
    return parsed_data

with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = parse(train_set)
    
# read test data
test_name = "test-public.txt"
with open(os.path.join(file_dir, test_name)) as f:
    test_set = f.readlines()
    test_set = parse(test_set)
    test_set.pop(0)

# turn ID into int
for i in range(0, len(test_set)):
    test_set[i][0] = int(test_set[i][0])

NameError: name 'parsed_data' is not defined

In [20]:
# Similarity

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

def getSigma(node:str, graph:dict) -> set:
    SigmaOut = getSigmaOut(node, graph)
    SigmaIn = getSigmaIn(node, graph)
    Sigma = SigmaIn.union(SigmaOut)
    return Sigma

def getSigmaOut(node:str, graph:dict) -> set:
    SigmaOut = set()
    try:
        for item in graph[node]:
            SigmaOut.add(item)
        return SigmaOut
    except:
        return SigmaOut

def getSigmaIn(node:str, graph:dict) -> set:
    SigmaIn = set()
    for key, value in graph.items():
        if node in value:
            SigmaIn.add(key)
    return SigmaIn

def cosine1(a:str, b:str, graph:dict) -> float:
    SigmaA = getSigma(a, graph)
    SigmaB = getSigma(b, graph)
    return float(len(SigmaA.intersection(SigmaB)) / (len(SigmaA)*len(SigmaB)))


def cosine2(a:str, b:str, graph:dict) -> float:
    score = 0
    SigmaOutA = getSigmaOut(a, graph)
    if SigmaOutA:
        for node in SigmaOutA:
            score += cosine1(b, node, graph)
        return float(score/len(SigmaOutA))
    else:
        return 0

In [11]:
# Cosine Type 1


train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]

Cosine1 = []
for ID, user, sink in test_set:
    Cosine1.append([ID, cosine1(user, sink, train_dic)])

# sort the results
Cosine1_result = sorted(Cosine1, key=lambda x: x[-1], reverse=True)
for item in Cosine1_result:
    index = Cosine1_result.index(item)
    if index < len(Cosine1_result)/2:
        Cosine1_result[index][-1] = 1
    else:
        Cosine1_result[index][-1] = 0
Cosine1_result = sorted(Cosine1_result, key=lambda x: x[0], reverse=False)

# output
title = ["Id", "Predicted"]
test_pd = pd.DataFrame(columns = title, data = Cosine1_result)
test_pd.to_csv('/Users/linzhizhang/Stu/SM4/90051/Cosine1.csv', encoding='utf-8')
test_pd = pd.DataFrame(columns = title, data = Cosine1)
test_pd.to_csv('/Users/linzhizhang/Stu/SM4/90051/Cosine1Score.csv',encoding='utf-8')

In [21]:
## Cosine type 2
import multiprocessing as mp
import time
import copy as cp


train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]
    
if __name__ == '__main__':
    
    def getPro(record:list) -> list:
        node = [record[0], cosine2(record[1], record[-1], train_dic)]
        print(node)
        return [record[0], cosine2(record[1], record[-1], train_dic)]
         
    # construct the # of pools corresponding to the cpu_count in ur PC
    pool = mp.Pool(mp.cpu_count())
    
    startTime = time.time()
    
    Cosine2 = pool.map(getPro, test_set)
    pool.close()
    pool.join()
    
    endTime = time.time()
    print("Total time:" + (endTime - startTime).__str__())
    
    Cosine2_result = sorted(Cosine2, key=lambda x: x[-1], reverse=True)
    Cosine2_result = cp.deepcopy(Cosine2_result)
    for item in Cosine2_result:
        index = Cosine2_result.index(item)
        if index < len(Cosine2_result)/2:
            Cosine2_result[index][-1] = 1
        else:
            Cosine2_result[index][-1] = 0
    Cosine2_result = sorted(Cosine2_result, key=lambda x: x[0], reverse=False)

    title = ["Id", "Predicted"]
    test_pd = pd.DataFrame(columns = title, data = Cosine2_result)
    test_pd.to_csv('/Users/linzhizhang/Stu/SM4/90051/Cosine2.csv',encoding='utf-8')
    test_pd = pd.DataFrame(columns = title, data = Cosine2)
    test_pd.to_csv('/Users/linzhizhang/Stu/SM4/90051/Cosine2Score.csv',encoding='utf-8')

[379, 0.000306850897406077]
[1, 0.0011086098198699584]
[2, 5.10988236771775e-05]
[380, 0.00046590642098970646]
[442, 0.0003432280747153645]
[381, 7.493616704833597e-05]
[127, 0.0005297209380207122]
[190, 0.0004393019593842166]
[316, 0.0002158649279589646]
[3, 0.000554185075143744]
[128, 0.0]
[443, 0.001261931787167804]
[129, 0.00016499858557873906]
[317, 0.0003368713628076996]
[130, 0.005015243610008192]
[318, 0.0038005978436400686]
[444, 6.210188668208815e-05]
[319, 0.000749219789548558]
[4, 0.0006345597575612522]
[131, 0.0004035221160847429]
[382, 0.008454155705113396]
[320, 0.0010513327829654708]
[445, 0.002163634830557858]
[132, 0.002500276904415105]
[5, 0.00032982896144428036]
[6, 0.0]
[191, 0.008316549176597087]
[446, 0.00037694535391118747]
[447, 0.0]
[448, 0.004538501008344911]
[7, 0.0004943337647365913]
[449, 9.407686682682286e-05]
[383, 0.004320262850029422]
[450, 7.391427777005935e-05]
[451, 0.0003123306878136951]
[8, 0.00010250772464213292]
[384, 0.0005804369562858412]
[452

Process ForkPoolWorker-28:
Process ForkPoolWorker-26:
Process ForkPoolWorker-25:


KeyboardInterrupt: 

Process ForkPoolWorker-29:
Process ForkPoolWorker-32:
Traceback (most recent call last):
Process ForkPoolWorker-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-30:
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-31:
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/linzhizhang/opt/anaconda3/lib/python3

  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-21-0537b428c73f>", line 14, in getPro
    node = [record[0], cosine2(record[1], record[-1], train_dic)]
  File "<ipython-input-20-409d585d6d6c>", line 40, in cosine2
    score += cosine1(b, node, graph)
  File "<ipython-input-20-409d585d6d6c>", line 30, in cosine1
    SigmaA = getSigma(a, graph)
  File "<ipython-input-20-409d585d6d6c>", line 9, in getSigma
    SigmaIn = getSigmaIn(node, graph)
  File "<ipython-input-20-409d585d6d6c>", line 25, in getSigmaIn
    if node in value:
KeyboardInterrupt


[631, 0.0020288804146068808]


Process ForkPoolWorker-42:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Process ForkPoolWorker-40:
Process ForkPoolWorker-38:
Process ForkPoolWorker-43:
Process ForkPoolWorker-39:
Process ForkPoolWorker-36:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Use

  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-21-0537b428c73f>", line 14, in getPro
    node = [record[0], cosine2(record[1], record[-1], train_dic)]
  File "<ipython-input-20-409d585d6d6c>", line 40, in cosine2
    score += cosine1(b, node, graph)
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-20-409d585d6d6c>", line 40, in cosine2
    score += cosine1(b, node, graph)
  File "/Users/linzhizhang/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **

In [ ]:
## Cosine type 3
def Cosine3(a:str, b:str, graph:dict) -> float:
    score = 0
    SigmaInB = getSigmaIn(b, graph)
    if SigmaInB:
        for node in SigmaInB:
            score += Cosine1(a, node, graph)
        return float(score/len(SigmaInB))
    else:
        return 0

train_dic = {}
for item in train_set:
    train_dic[item[0]] = item[1:]
    
Cosine3 = []
for ID, user, sink in test_set:
    Cosine3.append([ID, Cosine3(user, sink, train_dic)])
    
Cosine3_result = sorted(Cosine3, key=lambda x: x[-1], reverse=True)
for item in Cosine3_result:
    index = Cosine3_result.index(item)
    if index < len(Cosine3_result)/2:
        Cosine3_result[index][-1] = 1
    else:
        Cosine3_result[index][-1] = 0
Cosine3_result = sorted(Cosine3_result, key=lambda x: x[0], reverse=False)

title = ["Id", "Predicted"]
test_pd = pd.DataFrame(columns = title, data = Cosine3_result)
test_pd.to_csv('/Users/linzhizhang/Stu/SM4/90051/Cosine3.csv',encoding='utf-8')